In [2]:
# Cell 1 - Tell python where to find the modules to import from

import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline if notebook runs from the repo root or from notebooks/
cwd = Path.cwd()
project_root = cwd if (cwd / "src").exists() else cwd.parent # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print("Project root: ", project_root)

Project root:  /Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [4]:
# Cell 2 — Imports, Load & clean inputs (real paths)

from src import load_data
from src.clean_matrix import clean_matrix
from src.clean_relius_roth_basis import clean_relius_roth_basis
from src.build_correction_file import build_correction_dataframe, write_correction_file

from src.clean_relius_demo import clean_relius_demo

from src.config import RAW_DATA_DIR, USE_SAMPLE_DATA_DEFAULT


if USE_SAMPLE_DATA_DEFAULT:
    matrix_path = None
    relius_roth_basis_path = None
    relius_demo_path = None
    sheet_name = 0
else:
    matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"
    relius_roth_basis_path = RAW_DATA_DIR / "real_roth_basis_relius_2025.xlsx"
    relius_demo_path = RAW_DATA_DIR / "real_demo_relius_2025.xlsx"
    sheet_name = "Sheet5"


# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path)
matrix_clean = clean_matrix(matrix_raw)

# Load  and clean Relius Roth Basis raw data
relius_roth_basis_raw = load_data.load_relius_roth_basis_excel(path=relius_roth_basis_path, sheet_name=sheet_name)
relius_roth_basis_clean = clean_relius_roth_basis(relius_roth_basis_raw)

# Load and clean Relius Demo raw data
relius_demo_raw = load_data.load_relius_demo_excel(path=relius_demo_path)
relius_demo_clean = clean_relius_demo(relius_demo_raw)


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
print("matrix_raw DataFrame:    ", matrix_raw.shape)
print("matrix_clean DataFrame:  ", matrix_clean.shape)
print("matrix_clean 'ssn' dtype: ", matrix_clean["ssn"].dtype)
print("\n")

print("relius_roth_basis_raw:   ", relius_roth_basis_raw.shape)
print("relius_roth_basis_clean: ", relius_roth_basis_clean.shape)
print("relius_roth_basis_clean 'ssn' dtype: ", relius_roth_basis_clean["ssn"].dtype)
print("\n")
print("relius_demo_raw:       ", relius_demo_raw.shape)
print("relius_demo_clean:       ", relius_demo_clean.shape)
print("relius_demo_clean 'ssn' dtype: ", relius_demo_clean["ssn"].dtype)

matrix_raw DataFrame:     (8, 15)
matrix_clean DataFrame:   (7, 21)
matrix_clean 'ssn' dtype:  string


relius_roth_basis_raw:    (3, 6)
relius_roth_basis_clean:  (3, 11)
relius_roth_basis_clean 'ssn' dtype:  string


relius_demo_raw:        (5, 6)
relius_demo_clean:        (5, 11)
relius_demo_clean 'ssn' dtype:  string


In [5]:
# Cell 3 — Quick view of Relius Roth Basis clean data

print("relius_roth_basis_clean Head:")
relius_roth_basis_clean.head(10)


relius_roth_basis_clean Head:


,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt,ssn_valid,amount_valid,date_valid,code_1099r_valid,validation_issues
0,300005R,333445555,Mia,Chen,2016,10017.34,True,True,<NA>,<NA>,[]
2,300005R,666778888,Evan,Stone,1800,-100.00,False,False,<NA>,<NA>,"[ssn_invalid, amount_invalid]"
1,300005R,555667777,Zoe,Lopez,<NA>,6777.65,True,True,<NA>,<NA>,[]


In [6]:
# Cell 4 — Check key data inside Relius Roth Basis clean data

print("'ssn' lenght and index:\n", relius_roth_basis_clean["ssn"].str.len().value_counts(dropna=False))
print("\n'ssn' duplicates?:\n", relius_roth_basis_clean.duplicated(["plan_id", "ssn"]).sum())

'ssn' lenght and index:
 ssn
9    3
Name: count, dtype: Int64

'ssn' duplicates?:
 0


In [7]:
# Cell 5 — Test normalize_ssn function controls different types of formas from raw files

import pandas as pd
from src.normalizers import normalize_ssn

tests = pd.Series(["040511830", 40511830.0, "40511830.0", "040-51-1830"])
tests.map(normalize_ssn)

0    040511830
1    040511830
2    040511830
3    040511830
dtype: object

In [8]:
# Cell 6 — Check key SSNs that start with '0' to validate normalize function

ssn_zero_mask = (
    relius_roth_basis_clean["ssn"]
    .astype("string")
    .str.startswith("0")
    .fillna(False)
)
print(f"SSN count that starts with '0': {relius_roth_basis_clean.loc[ssn_zero_mask].shape[0]}")
relius_roth_basis_clean[relius_roth_basis_clean["ssn"].str.startswith("0")].head(10)

SSN count that starts with '0': 0


,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt,ssn_valid,amount_valid,date_valid,code_1099r_valid,validation_issues


In [9]:
# Cell 7 — Check column(s) that are Python 'list'

list_cols = [
    col for col in relius_roth_basis_clean.columns
    if relius_roth_basis_clean[col].apply(lambda x: isinstance(x, list)).any()
]

list_cols

['validation_issues']

Notes:
- Code Crashed in Cell #6 -> `print(relius_roth_basis_clean[relius_roth_basis_clean["ssn"].str.startswith("0").fillna(False)].value_counts().sum())`
    - since there is a column that stores a List (not string, float, ints, dates) and Python `list`is not hashable.

In [10]:
# Cell 8 — Check column´s Dtypes in relius_roth_basis_clean DataFrame

relius_roth_basis_clean.dtypes

plan_id                string[python]
ssn                    string[python]
first_name             string[python]
last_name              string[python]
first_roth_tax_year             Int64
roth_basis_amt                float64
ssn_valid                     boolean
amount_valid                  boolean
date_valid                    boolean
code_1099r_valid              boolean
validation_issues              object
dtype: object

In [11]:
# Cell 9 — Validate required columns exist (pre-flight)

required_matrix_cols = {
    "plan_id","ssn","txn_date","transaction_id","participant_name","matrix_account",
    "gross_amt","fed_taxable_amt","roth_initial_contribution_year","tax_code_1","tax_code_2"
}

# The '-' operator between sets is set difference
# “Give me all items that are in required_matrix_cols but not in matrix_clean.columns.”
missing = required_matrix_cols - set(matrix_clean.columns)

# assert 'CONDITION', "error message if condition is False"
# if it's True  -> nothing happens; code continues normally.
# if it's False -> Python raises an 'AssertionError' with the provided message.
assert not missing, f"Matrix missing columns: {missing}"

required_demo_cols = {"plan_id","ssn","dob"}
missing = required_demo_cols - set(relius_demo_clean.columns)
assert not missing, f"Demo missing columns: {missing}"

required_basis_cols = {"plan_id","ssn","first_roth_tax_year","roth_basis_amt"}
missing = required_basis_cols - set(relius_roth_basis_clean.columns)
assert not missing, f"Roth basis missing columns: {missing}"

print("✓ Required columns present")

✓ Required columns present


In [12]:
# Cell 10 — Run Roth Basis Taxable Analysis Engine

from src.roth_taxable_analysis import run_roth_taxable_analysis


relius_roth_basis = run_roth_taxable_analysis(
    matrix_clean,
    relius_demo_clean,
    relius_roth_basis_clean
)

print("relius_roth_basis_df:", relius_roth_basis.shape)
relius_roth_basis.head(10)

relius_roth_basis_df: (2, 23)


,transaction_id,txn_date,ssn,participant_name,matrix_account,plan_id,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,...,roth_initial_contribution_year,first_roth_tax_year,start_roth_year,roth_basis_amt,age_at_txn,suggested_taxable_amt,suggested_first_roth_tax_year,correction_reason,action,match_status
0,2032640,2024-03-07,333445555,Mia Chen,07E00442,300005R,B,G,H,<NA>,...,2016,2016,2016,10017.34,39.0,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- taxable_w...,UPDATE_1099\nINVESTIGATE,match_needs_correction
1,2032640,2024-03-08,333445555,Mia Chen,07G00442,300005R,B,<NA>,B,1,...,<NA>,2016,2016,10017.34,39.0,<NA>,2016,- roth_initial_year_mismatch\n- roth_age_tax_c...,UPDATE_1099,match_needs_correction


In [13]:
# Cell 11 — Output schema check (builder-compatible canonical fields)

required_out_cols = {
    "transaction_id","txn_date","ssn","participant_name","matrix_account",
    "tax_code_1","tax_code_2","suggested_tax_code_1","suggested_tax_code_2",
    "correction_reason","action","match_status",
    "suggested_taxable_amt","suggested_first_roth_tax_year"
}
missing = required_out_cols - set(relius_roth_basis.columns)
assert not missing, f"Engine C output missing columns: {missing}"

print("✓ Engine C output schema OK (builder-compatible)")

✓ Engine C output schema OK (builder-compatible)


In [14]:
# Cell 12 — Filter validation (Roth-only + inherited excluded)

# Roth plan check based on plan_id rules:
is_roth = relius_roth_basis["plan_id"].astype(str).str.startswith("300005") | relius_roth_basis["plan_id"].astype(str).str.endswith("R")
assert is_roth.all(), "Found non-Roth plan_id rows in Engine C output." # .all() returns True only if every value in the Series is True.
                                                                        # If at least one row is False -> .all() returns False.

print("✓ Roth-only filter passed")

✓ Roth-only filter passed


In [15]:
# Cell 13 — Join coverage diagnostics (DOB + basis availability)

import pandas as pd

print("DOB missing in Engine C output:", relius_roth_basis.get("dob", pd.Series(dtype=object)).isna().sum() if "dob" in relius_roth_basis.columns else "DOB not retained")
print("first_roth_tax_year missing:", relius_roth_basis["suggested_first_roth_tax_year"].isna().sum(), "(note: this can be NA if not needed)")

DOB missing in Engine C output: DOB not retained
first_roth_tax_year missing: 1 (note: this can be NA if not needed)


In [16]:
# Cell 14 — Validate “basis coverage” rule is actually triggering

zero_taxable = relius_roth_basis[relius_roth_basis["suggested_taxable_amt"].fillna(pd.NA).eq(0.0)]
print("Rows suggesting taxable=0:", len(zero_taxable))
zero_taxable[
    [
        "plan_id","ssn","age_at_txn","gross_amt","fed_taxable_amt",
        "roth_initial_contribution_year","first_roth_tax_year", "roth_basis_amt",
        "suggested_first_roth_tax_year","suggested_taxable_amt","correction_reason","match_status","action"
    ]
].head(25)

Rows suggesting taxable=0: 0


,plan_id,ssn,age_at_txn,gross_amt,fed_taxable_amt,roth_initial_contribution_year,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action


Notes:
- This checks that suggested_taxable_amt == 0 is being produced and why.
- Already fixed: 
    - if 'roth_basis_amt' > 'gross_amount' AND 'first_roth_tax_year' == 'roth_initial_contribution_year' -> 'no correction needed' or 'qualified_roth_distribution'
    - if ppt is older than 59 1/2 AND 'current year' >= 'roth_initial_contribution_year + '5 years' -> 'no correction needed' or 'qualified_roth_distribution'
    - if 'first_roth_tax_year' != 'roth_initial_contribution_year' -> needs_correction

In [17]:
# Cell 15 — Validate the 15% proximity flag (INVESTIGATE behavior)

investigate_df = relius_roth_basis[relius_roth_basis["action"].eq("INVESTIGATE")]
print("INVESTIGATE rows:", len(investigate_df))
investigate_df[["plan_id","ssn","gross_amt","fed_taxable_amt","correction_reason","match_status","action"]].head(25)

INVESTIGATE rows: 0


,plan_id,ssn,gross_amt,fed_taxable_amt,correction_reason,match_status,action


In [18]:
# Cell 16 - Validate columns for quick export to Excel to present to stakeholders

action_df = relius_roth_basis[relius_roth_basis["action"].notna()]
print(f"NEED ACTION rows: {len(action_df)}")
export_roth_basis_df = action_df[
    [
        "plan_id","ssn","participant_name","age_at_txn", "tax_code_1",
        "tax_code_2", "suggested_tax_code_1",
        "suggested_tax_code_2", "new_tax_code", "gross_amt","fed_taxable_amt",
        "roth_initial_contribution_year","first_roth_tax_year","roth_basis_amt",
        "suggested_first_roth_tax_year","suggested_taxable_amt","correction_reason",
        "match_status","action", "matrix_account", "transaction_id", "txn_date",
    ]
]

export_roth_basis_df.head(15)

NEED ACTION rows: 2


,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,333445555,Mia Chen,39.0,B,G,H,<NA>,H,10930.8,...,2016,10017.34,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- taxable_w...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,2032640,2024-03-07
1,300005R,333445555,Mia Chen,39.0,B,<NA>,B,1,B1,4505.9,...,2016,10017.34,2016,<NA>,- roth_initial_year_mismatch\n- roth_age_tax_c...,match_needs_correction,UPDATE_1099,07G00442,2032640,2024-03-08


In [19]:
# Cell 17 - Validate engine behavior for tax codes 'B' and 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("B") & export_roth_basis_df["tax_code_2"].eq("G")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,333445555,Mia Chen,39.0,B,G,H,<NA>,H,10930.8,...,2016,10017.34,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- taxable_w...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,2032640,2024-03-07


In [20]:
# Cell 18 - Validate engine behavior for tax codes '4' and 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("4") & export_roth_basis_df["tax_code_2"].eq("G")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [21]:
# Cell 19 - Validate engine behavior for tax codes 'B' and '4'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("B") & export_roth_basis_df["tax_code_2"].eq("4")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [22]:
# Cell 20 - Validate engine behavior for tax code '4'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("4")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [23]:
# Cell 21 - Validate engine behavior for tax code 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_2"].eq("G")].head(10)

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,333445555,Mia Chen,39.0,B,G,H,<NA>,H,10930.8,...,2016,10017.34,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- taxable_w...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,2032640,2024-03-07


'--------------------------------  Test Quick Export to Excel File --------------------------------'

In [ ]:
# Cell 22 — Use quick report export for manual DataFrame output to Excel for stakeholders

from src.export_utils import write_df_excel

path = write_df_excel(export_roth_basis_df, filename_prefix="export_roth_distribs", engine="roth_taxable")

print(f"Export was successful!\nFile path: {path}")